# Báo cáo Đồ án Song song hóa và tối ưu hóa một ứng dụng
- Thông tin nhóm
  - Lê Thanh Hiếu - 1712434
  - Nguyễn Văn Tuấn - 1512638
  - Phan Hữu Tú - 1712861
- Github: https://github.com/tuancamtbtx/gpu-python

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import os
os.chdir("/content/drive/MyDrive/LTSSUD/gpu-python")

## 1. Mô tả ứng dụng
- **Ứng dụng**: **Thay đổi kích thước ảnh bằng Seam Carving**
    - **Input**: một tấm ảnh (RGB)
    - **Output**: một tấm ảnh mới được thay đổi kích thước (không chỉ cho chiều rộng mà còn cho chiều cao, không chỉ thu nhỏ mà còn phóng to) từ tấm ảnh gốc mà không làm biến dạng các đối tượng quan trọng.
- **Ý nghĩa thực tế khi sử dụng?**
    - Khi muốn có các phiên bản với các kích thước khác nhau của cùng một tấm ảnh để hiển thị trên các thiết bị khác nhau (máy tính, điện thoại, ...)
   - Một ví dụ thực tế khi tạo thumbnail (ảnh đại diện) cho 1 video chúng ta sẽ dựa vào nhưng yếu tố khác để resize ảnh về đúng khung hình ta mong muốn như màn hình điện thoại 16:9 , 3:4

- **Ứng dụng này có cần tăng tốc không?**  
    - Vì seam carving rất chậm khi cài đặt tuần tự và có tiềm năng song song hóa.<br/>
    ![](readmeimages/seam_carving.png) <br/>
> Nguồn ảnh: Slides Lập trình song song HK1


## 2. Cài đặt tuần tự


### 2.1. Thiết kế
- **Ý tưởng**: 
    - Một seam: một tập các pixel, mỗi dòng một pixel,pixel của dòng r & dòng r+1 được kết nối với nhau
    - Việc thay đổi kích thước ảnh được thực hiện bằng cách loại bỏ đi các điểm ảnh ít quan trọng và giữ lại các điểm ảnh quan trọng.


#### 2.1.1. Các bước triển khai thuật toán

![](readmeimages/workflow.png)

##### Thuật toán xóa seam theo cột
  - Bước 1: Chuyển ảnh từ RGB sang Grayscale
  - Bước 2: Tính bảng Energy của ảnh
  - Bước 3: Tìm bảng chi phí nhỏ nhất
  - Bước 4: Tìm seam có chi phí nhỏ nhất
  - Bước 5: Xóa seam nhỏ nhất tìm được ở bước 4
  - Bước 6: Lặp lại bước 5 cho tới khi xóa  đủ số seams cần xóa


##### Thuật toán xóa seam theo hàng
  - Bước 1: Xoay ảnh một góc 90 độ theo chiều kim đồng hồ
  - Bước 2: Lặp lại thuật toán xóa seam theo cột cho đến khi xóa đủ số seams cần xóa
  - Bước 3: Xoay ảnh lại một góc 90 độ ngược chiều kim đồng hồ

##### Thuật toán chèn seam theo cột
  - Bước 1: Thực hiện thuật toán xóa seam theo cột trên một ảnh được sao chép từ ảnh gốc và lưu lại danh sách các điểm ảnh đã xóa theo đúng thứ tự đã xóa
  - Bước 2: Chèn các seams vào ảnh gốc theo đúng thứ tự tọa độ đã lưu. Với giá trị của pixel được chèn được lấy bằng trung bình của 2 pixels kế bên.

##### Thuật toán chèn seam theo hàng
  - Bước 1: Xoay ảnh 90 độ theo chiều kim đồng hồ
  - Bước 2: Thực hiện thuật toán chèn seam theo cột cho đến khi đủ số seams cần xóa
  - Bước 3: Xoay ảnh lại một góc 90 độ ngược chiều kim đồng hồ

#### 2.1.2. Chi tiết thuật toán backward energy

##### Xây dựng bảng Energy
- Chuyển ảnh từ rgb sang graysale
- Dùng edge detetion
    - Phát hiện ảnh theo chiều x (1): thực hiện convolution giữa ảnh grayscale với bộ lọc x-Sobel
    - Phát hiện ảnh theo chiều y (2): thực hiện convolution giữa ảnh grayscale với bộ lọc y-Sobel
- Độ quan trọng của mỗi pixel = abs(kết quả tương ứng của (1)) + abs(kết quả tương ứng của (2))

##### Xây dựng bảng chi phí nhỏ nhất
- Sử dụng Dynamic Programming để xây dựng bảng chi phí nhỏ nhất
    - Ở hàng trên cùng ta có tổng chi phí giữ nguyên. Với các hàng còn lại, chi phí của mỗi pixel bằng engergy của nó cộng với chi phí nhỏ nhất của 3 pixel liền kề phía trên. <br/>
    ![](readmeimages/algo1.png)<br/>
    > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project

    - Lặp lại quá trình cho đến hàng cuối cùng ta được bảng chi phí nhỏ nhất. <br/>
    ![](readmeimages/algo2.png)
    > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project


##### Xác định seam nhỏ nhất
- Từ bảng chi phí nhỏ nhất trên ta xác định được vị trí có tổng chi phí nhỏ nhất ở hàng cuối cùng và quay lui để xác định seam nhỏ nhất<br/>
![](readmeimages/algo3.png)
> Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project

#### 2.1.3 Chi tiết thuật toán forward energy (cải tiến backward energy)

##### Ý tưởng cải tiến
- Forward Energy dự đoán những điểm ảnh nào sẽ liền kề sau khi loại bỏ đường seam và sử dụng điều đó để đề xuất đường seam tốt nhất để loại bỏ. Điều này trái ngược với backward energy.

##### Tại sao phải cải tiến?

   ![](readmeimages/backwards-example.png)

- Đối với thuật toán cũ thì công thức tính toán năng lượng nhỏ nhất của 1 pixel dựa vào 3 pixel liền kề phía trên. 
    ![](readmeimages/differents.png)
    > Nguồn ảnh: https://avikdas.com/2019/07/29/improved-seam-carving-with-forward-energy.html

- Điều này sẽ gây trở ngại lớn nếu cả 3 pixel có độ chênh lệch không lớn, và dẫn tới việc loại bỏ seam (điểm có năng lượng nhỏ nhất) thì lại vô tình bỏ đi phần quan trọng của texture trong tấm hình.


##### Mô tả thuật toán
Hình dưới là mô tả rõ nét giữa 2 seams liên tiếp nằm trong 1 đường seam của hình. Như vậy, chúng ta sẽ có 3 trường hợp như trong hình, 
- seam (x-1, y-1) liên kết seam (x, y)
- seam (x, y-1) liên kết seam (x, y)
- seam (x+1, y-1) liên kết seam (x, y)
    ![](readmeimages/seam-removal-new-edges.png)
    > Nguồn ảnh: https://avikdas.com/2019/07/29/improved-seam-carving-with-forward-energy.html

Tiếp theo, chúng ta phải xem xét những pixel nào được kết hợp với nhau bằng cách loại bỏ một pixel cụ thể. Điều này phụ thuộc vào việc pixel hiện tại được kết nối với một đường seam ở trên cùng bên trái (top-left), trên cùng (top-center) hoặc trên cùng bên phải (top-right). 

Ví dụ: nếu pixel hiện tại (x, y) kết nối với đường nối ở trên cùng bên trái (x - 1, y - 1), thì các biên cạnh mới được tạo thành do sự giao nhau mới giữa các pixel:

- Pixel ở bên trái của pixel hiện tại, với tọa độ (x-1, y), hiện đang chạm vào pixel ở bên phải của pixel hiện tại, có tọa độ (x + 1, y).

- Pixel phía trên pixel hiện tại, có tọa độ (x, y − 1), hiện đang chạm vào pixel ở bên trái của pixel hiện tại, có tọa độ (x − 1, y).

Vì các cặp pixel này sẽ chạm nhau sau khi loại bỏ đường seam, ta sẽ so sánh sự khác biệt về màu sắc giữa mỗi cặp pixel. Lưu ý rằng chỉ xem xét các cạnh mới liên quan đến pixel trong hàng hiện tại, vì hàng trước đó được tính (do thực hiện tuần tự từng hàng).



##### Các công thức thực hiện
- Ta quy định lần lượt: L = top-left, U = top-center, R = top-right

- Công thức tính độ chênh lệch giữa 2 điểm ảnh bất kỳ (grayscale):
<br>

$$D(x, y) = \lvert M_{p_0} - M_{p_1} \rvert$$



- Công thức tính độ chênh lệch giữa các cặp pixel tạo ra biên cạnh mới (phục vụ quy hoạch động):

$C_{L}(x, y) = D[(x - 1, y), (x + 1, y)] + D[(x, y - 1), (x - 1, y)]$
<br>
$C_{U}(x, y) = D[(x - 1, y), (x + 1, y)]$
<br>
$C_{R}(x, y) = D[(x - 1, y), (x + 1, y)] + D[(x, y - 1), (x + 1, y)]$



- Không giống như trước đây, mỗi lựa chọn seam để tiếp tục kết nối các pixel khác nhau lại với nhau nó thuộc đường seam. Điều này có nghĩa là ta phải xem xét 3 chi phí khác nhau trong mỗi trường hợp $(L, U, R)$. 
<br>
    - Ví dụ: nếu tiếp tục đường seam là từ pixel trên cùng bên trái (L), ta phải xem xét chi phí $C_L$ làm giá trị năng lượng tại pixel đang xét. Cuối cùng, ta chọn tùy chọn năng lượng thấp nhất trong số tất cả các tùy chọn này với công thức quy hoạch động:
    
$$ M(x, y) = min( \begin{cases} M(x - 1, y - 1) + C_{L}(x, y)  \\  M(x, y - 1) + C_{U}(x, y) \\  M(x + 1, y - 1) + C_{R}(x, y) \end{cases} )$$


- Từ công thức này, ta có thể thấy, mỗi bước forward energy đều dự đoán được seam tiếp theo dựa trên việc dự đoán khi bỏ seam đó đi thì các biên cạnh mới đc tạo thành độ kết nối giữa các cặp pixel lớn hay nhỏ.


### 2.2. Đánh giá

#### 2.2.1 Đánh giá về kết quả

##### Kết quả giữa hàm backward energy và forward energy

| backward_energy | forward_energy|
|:---:|:---:|
|<img src="images/ouput_energy.jpg" width="350">|<img src="images/ouput_forward_energy.jpg" width="350">|

##### Kết quả theo số seam

| Số seams |                  Xóa theo cột                   |                Xóa theo hàng                 |                  Chèn theo cột                  |                Chèn theo hàng                 |
| :------: | :---------------------------------------------: | :------------------------------------------: | :---------------------------------------------: | :-------------------------------------------: |
|   100    | ![](images/output_remove100seams_by_column.jpg) | ![](images/output_remove100seams_by_row.jpg) | ![](images/output_remove100seams_by_column.jpg) | ![](images/output_insert100seams_by_rows.jpg) |
|   300    | ![](images/output_remove300seams_by_column.jpg) | ![](images/output_remove300seams_by_row.jpg) | ![](images/output_insert300seams_by_column.jpg) | ![](images/output_insert300seams_by_rows.jpg) |

__So sánh kết quả hàm forward energy với bài báo gốc__

| Ảnh gốc | Results (paper) | Forward Energy (nhóm tự cài đặt)|
|:---:|:---:|:---:|
|![](readmeimages/rain_origin.png)|![](readmeimages/rain_result_paper.png)|![](readmeimages/rain_remove150seams_by_column.jpg)|

__Nhận xét__: Phiên bản tự cài đặt của nhóm cho kết quả khá tốt, gần như tương đương so với paper gốc

#### 2.2.2. Đánh giá về thời gian

##### Thời gian giữa hàm backward energy và forward energy

Thời gian tính backward energy và forward energy với input là bức ảnh có kích thước 968x1428

|backward energy|forward energy|
|:---:|:---:|
|1.247s |0.731s |

##### Thời gian chạy theo số seam

| Số seams | Xóa theo cột | Xóa theo hàng | Chèn theo cột | Chèn theo hàng |
| :------: | :----------: | :-----------: | :-----------: | :------------: |
|   100    |   26.952s    |    26.437s    |    28.313s    |    27.982s     |
|   300    |   71.227s    |    67.611s    |    80.591s    |    78.045s     |
|   600    |   125.444s   |    108.591s   |   140.174s    |    129.985s    |

Tổng thời gian: 914.206s

## 3. Cài đặt song song

### 3.1. Phân tích

#### 3.1.1. Chọn các thành phần song song hóa

- Nhóm tiến hành chạy phiên bản tuần tự chèn 100 seams theo chiều ngang để in ra thời gian chạy của các hàm

In [ ]:
!python seam_carving_cpu.py -in "images/input.jpg" -out "output_cpu/output_insert100seams_by_row.jpg" -dx 0 -dy 100 -test_time

SEAM CARVING CPU
Input image shape: (968, 1428, 3)
rgb2gray time: 1.056 seconds
forward energy time:  41.489 seconds
get minimum cost table time: 0.924 seconds
get minimum seam time:  0.280 seconds
remove seam time: 1.125 seconds
insert seam time: 2.302 seconds
Output image shape: (1068, 1428, 3)
Total time: 47.390 seconds


Nhận thấy rằng thời gian chạy của các hàm ngoài get_minimum_seam đều có thời gian lớn và có tiềm năng song song hóa (các pixels độc lập), đặc biệt là hàm forward energy. Nên nhóm quyết định song song hóa các hàm rgb2gray, forward_energy, get_minimum_cost _table, remove_seam và insert_seam

### 3.2. Thiết kế

#### 3.2.1. Chuyển ảnh RGB sang ảnh xám trên hàm Kernel 

- Mỗi thread sẽ chịu trách nhiệm tính kết quả grayscale cho một pixel.

#### 3.2.2. Tính bảng năng lượng trên hàm Kernel 

Vì hàm forward energy có sử  dụng một bảng tích lũy năng lượng để tính toán các đường nối bằng quy hoạch động nên kết quả dòng phía dưới phải dựa vào dòng phía trên. Từ đây ta sẽ song song hóa theo từng dòng, với mỗi cột trong một dòng sẽ được đảm nhận bởi một thread

#### 3.2.3. Xây dựng bảng chi phí nhỏ nhất trên hàm Kernel 

Để tìm bảng chi phí nhỏ nhất.  
- Vì chi phí nhỏ nhất của mỗi pixel đươc tính bằng cách cộng thêm chi phí nhỏ nhất của 1 trong 3 pixel kề trên nên ta có thể song song hóa việc tính toán theo từng dòng.
- Khi gọi hàm kernel này, mỗi thread sẽ được tạo cho mỗi column, bảng chi phí nhỏ nhất được tạo theo từng dòng, với mỗi thread sẽ tính chi phí cho một pixel trên dòng đó.  
- Vì mỗi dòng của bảng chi phí nhỏ nhất sẽ phụ thuộc vào dòng phía trên, những threads sẽ phải synchronize sau mỗi dòng.  

#### 3.2.4. Xóa seam trên hàm Kernel 

- Đối với hàm xóa seams thì các xử lý pixels của ảnh là độc lập, từ đó ta có thể tối ưu hóa bằng hàm kernel được áp dụng trên từng điểm ảnh của hình.
- Việc xóa seams bản chất là gán lại giá trị của từng pixels trên ảnh gốc sang ảnh kết quả, các phép gán độc lập có thể được thực hiện trên từng threads.


#### 3.2.5. Chèn seam trên hàm Kernel 

- Tương tự với xóa seams, các pixels của ảnh là độc lập, từ đó ta có thể tối ưu hóa bằng hàm kernel được áp dụng trên từng điểm ảnh của hình.
- Việc chèn seams bản chất là gán lại giá trị của từng pixels trên ảnh gốc sang ảnh kết quả, các phép gán độc lập có thể được thực hiện trên từng threads.


### 3.3. Đánh giá 

#### 3.3.2. Đánh giá độ chính xác 

MÔ TẢ CHO HÀM HASH IMAGES

Mỗi bức hình được chạy ra ở phiên bản cpu và phiên bản gpu được lưu ở 2 folder tương ứng (1 folder chứa output gpu, 1 folder chứa output cpu) và 2 tấm hình output so sánh phải có cùng tên.

Hàm hash sử dụng thư viện `hashlib` của python.
Hash được thực hiện trên `sha256` cho từng bức hình và so sánh giá trị này của từng bức hình để đưa ra kết quả.

Nếu hash của 2 bức hình giống nhau, in kết quả ra file `compare_images_result.txt` với số lượng hình giống nhau và khác nhau ở cả 2 folder.

In [ ]:
!python test_hash_images.py output_cpu output_gpu

Comparing output_cpu/remove200seams_by_row_baseline.jpg & output_gpu/remove200seams_by_row_baseline.jpg
Status: PASS
Image 1(sha256): 13a37fc754974b59233d97a95ca8f5b8405ca26d29ab110c448a52e900c1c69b
Image 2(sha256): 13a37fc754974b59233d97a95ca8f5b8405ca26d29ab110c448a52e900c1c69b
Comparing output_cpu/insert500seams_by_row_baseline.jpg & output_gpu/insert500seams_by_row_baseline.jpg
Status: PASS
Image 1(sha256): 3b32cfaca8875ddf23ec5b40c5d7958acd52165d3ee657a916f7769dcd5e3c56
Image 2(sha256): 3b32cfaca8875ddf23ec5b40c5d7958acd52165d3ee657a916f7769dcd5e3c56
Comparing output_cpu/insert200seams_by_column_baseline.jpg & output_gpu/insert200seams_by_column_baseline.jpg
Status: PASS
Image 1(sha256): eb1476fa05c7eb367cf4c12405ad6c45a9e76a3f8f9c8fa517d66bb4c762478e
Image 2(sha256): eb1476fa05c7eb367cf4c12405ad6c45a9e76a3f8f9c8fa517d66bb4c762478e
Comparing output_cpu/insert200seams_by_row_baseline.jpg & output_gpu/insert200seams_by_row_baseline.jpg
Status: PASS
Image 1(sha256): ce3b5800f32da757e

#### 3.3.3. Đánh giá về thời gian chạy

Sử dụng nvprof để đo thời gian chạy các hàm kernel

In [ ]:
!nvprof --normalized-time-unit s python seam_carving_gpu.py -in "images/input.jpg" -out "images/output_insert100seams_by_row_gpu.jpg" -dx 0 -dy 100

SEAM CARVING GPU - BASELINE
Input image shape: (968, 1428, 3)
==421== NVPROF is profiling process 421, command: python3 seam_carving_gpu.py -in images/input.jpg -out images/output_insert100seams_by_row_gpu.jpg -dx 0 -dy 100
Output image shape: (1068, 1428, 3)
Total time: 61.459 seconds
==421== Profiling application: python3 seam_carving_gpu.py -in images/input.jpg -out images/output_insert100seams_by_row_gpu.jpg -dx 0 -dy 100
==421== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    38.61  1.702567       500  3.41e-03  2.56e-06  0.012336  [CUDA memcpy DtoH]
                    31.10  1.371629       400  3.43e-03  1.73e-06  8.23e-03  [CUDA memcpy HtoD]
                    16.54  0.729433    142700  5.11e-06  5.02e-06  6.08e-06  cudapy::__main__::forward_energy_kernel$242(Array<double, int=2, C, mutable, aligned>, Array<double, int=2, C, mutabl

Nhận xét: Phiên bản GPU baseline chạy trên colab cho kết quả chưa tốt.  
Nguyên nhân: Hàm kernel chiếm chi phí lớn nhất là hàm forward energy nhưng ở phương pháp forward nó lại phải thực hiện theo từng dòng dẫn đến số lượng threads song song hóa không đủ nhiều.

## 4. Tối ưu hóa cài đặt song song


### 4.1. Tối ưu hóa 1: Sử dụng SMEM


#### 4.1.1. Phân tích
- Nhận thấy các hàm forward energy, get_minimum_cost_table đọc dữ liệu từ GMEM tương đối nhiều lần nên nhóm tiến hành cài đặt lại các hàm kernel này sử dụng SMEM để giảm truy xuất GMEM


#### 4.1.2. Cài đặt


##### 4.1.2.1. Cài đặt SMEM cho hàm forward energy kernel
- Do mỗi lần gọi kernel chỉ sử dụng 2 row (r và r-1) trong mảng ảnh xám nên ta chỉ cần khởi tạo SMEM có kích thước (BLOCK_SIZE + 2, 2) cho mảng, BLOCK_SIZE + 2 là để lưu thêm 2 giá trị ngoài cùng của mỗi dòng, tránh truy xuất ngoài bound của mảng trong phần tính toán.
- Với mảng m phục vụ quy hoạch động, hàm kernel sử dụng dòng r-1 phía trên để tìm giá trị năng lượng. Cũng tương tự như trên ta phải khởi tạo SMEM với kích thước BLOCK_SIZE + 2 để tránh truy xuất index sai.

##### 4.1.2.1. Cài đặt SMEM cho hàm get minimum cost table kernel


- Mỗi lần gọi hàm kernel sẽ truy xuất dòng r-1 trong mảng energy nên ta sử dụng SMEM cho dòng này. Cấp phát mảng có kích thước BLOCK_SIZE + 2 để tránh truy xuất index sai.

#### 4.1.3. Đánh giá


##### 4.1.3.1. Đánh giá về độ chính xác


In [ ]:
!python test_hash_images.py output_gpu output_gpu_v1

Comparing output_gpu/remove50seams_by_column_baseline.jpg & output_gpu_v1/remove50seams_by_column_baseline.jpg
Status: FAIL
Image 1(sha256): 6b325a02436ccf3f90281b9725b9b1d2725c1f7a9cda9b057d02f04923a92cdb
Image 2(sha256): 0f769828d65377a17537c795075eb1b1af4af6df8d2d5b5179cddb113e589d9f
Comparing output_gpu/insert200seams_by_row_baseline.jpg & output_gpu_v1/insert200seams_by_row_baseline.jpg
Status: FAIL
Image 1(sha256): ce3b5800f32da757e9333ff959c9e1a158d4c6592a2cf6ae3b6fb5d2c66a81e6
Image 2(sha256): 7af9f6ac8c3d3963f98fecff8b604c886e03e0dde32a688f41272dd1667bec9d
Comparing output_gpu/insert200seams_by_column_baseline.jpg & output_gpu_v1/insert200seams_by_column_baseline.jpg
Status: FAIL
Image 1(sha256): eb1476fa05c7eb367cf4c12405ad6c45a9e76a3f8f9c8fa517d66bb4c762478e
Image 2(sha256): 22a896ba11eb9fa25eead06ac7220dc21da83d67c885b9f78f8f298bd389122c
Comparing output_gpu/insert50seams_by_column_baseline.jpg & output_gpu_v1/insert50seams_by_column_baseline.jpg
Status: FAIL
Image 1(sha256

Nguyên nhân: Do lỗi ở phần tính forward energy hàm kernel mới (nhóm chưa sửa được lỗi này)

##### 4.1.3.2. Đánh giá về thời gian


In [ ]:
!nvprof --normalized-time-unit s python seam_carving_gpu_v1.py -in "images/input.jpg" -out "images/output_insert100seams_by_row_gpu_v1.jpg" -dx 0 -dy 100

SEAM CARVING GPU - OPTIMIZE 1
Input image shape: (968, 1428, 3)
==469== NVPROF is profiling process 469, command: python3 seam_carving_gpu_v1.py -in images/input.jpg -out images/output_insert100seams_by_row_gpu_v1.jpg -dx 0 -dy 100
Output image shape: (1068, 1428, 3)
Total time: 61.335 seconds
==469== Profiling application: python3 seam_carving_gpu_v1.py -in images/input.jpg -out images/output_insert100seams_by_row_gpu_v1.jpg -dx 0 -dy 100
==469== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    36.81  1.703547       500  3.41e-03  2.56e-06  0.012241  [CUDA memcpy DtoH]
                    29.73  1.375683       400  3.44e-03  1.76e-06  9.18e-03  [CUDA memcpy HtoD]
                    18.18  0.841272    142700  5.90e-06  5.60e-06  6.88e-06  cudapy::__main__::forward_energy_kernel_v1$242(Array<double, int=2, C, mutable, aligned>, Array<double,

Nhận xét: Thời gian chạy của phiên bản tối ưu 1 giảm khoảng 30% ở hàm forward energy kernel nhưng lại chậm hơn ở hàm get_minimum_cost_table. Dẫn đến tổng thời gian chạy không khác biệt nhiều so với phiên bản GPU.  

Nguyên nhân: Có thể là do "bank conflict" nên việc sử dụng SMEM chưa được tối ưu

### 4.2. Tối ưu hóa 2: Thử nghiệm nhiều blocksize


#### 4.2.1. Phân tích


Nhận xét: Các block size có thể sẽ thay đổi thời gian chạy do block size quyết định số  lượng thead trong 1 block.
Việc chọn block size thì rất khó để đánh giá vì phụ thuộc vào cách hiện thực chương trình và phần cứng là GPU, do đó có thể chọn nhiều block size và chạy thử để quyết định

Reference: https://stackoverflow.com/questions/9985912/how-do-i-choose-grid-and-block-dimensions-for-cuda-kernels

#### 4.2.2. Cài đặt


In [20]:
import os
os.environ['BLOCK_SIZE_1D'] = '64'
os.environ['BLOCK_SIZE_2D'] = '32'

In [21]:
!python seam_carving_gpu.py -in "images/input.jpg" -out "output_gpu/insert200seams_by_row_baseline.jpg" -dx 0 -dy 200 -checksum

SEAM CARVING GPU - BASELINE
Input image shape: (968, 1428, 3)
Output image shape: (1168, 1428, 3)
Total time: 106.233 seconds


In [22]:
!python seam_carving_gpu_v1.py -in "images/input.jpg" -out "output_gpu_v1/insert200seams_by_column_baseline.jpg" -dx 200 -dy 0 -checksum

SEAM CARVING GPU - OPTIMIZE 1
Input image shape: (968, 1428, 3)
Output image shape: (968, 1628, 3)
Total time: 77.461 seconds


In [23]:
os.environ['BLOCK_SIZE_1D'] = '64'
os.environ['BLOCK_SIZE_2D'] = '32'

In [24]:
!python seam_carving_gpu.py -in "images/input.jpg" -out "output_gpu/insert200seams_by_row_baseline.jpg" -dx 0 -dy 200 -checksum

SEAM CARVING GPU - BASELINE
Input image shape: (968, 1428, 3)
Output image shape: (1168, 1428, 3)
Total time: 107.464 seconds


In [25]:
!python seam_carving_gpu_v1.py -in "images/input.jpg" -out "output_gpu_v1/insert200seams_by_column_baseline.jpg" -dx 200 -dy 0 -checksum

SEAM CARVING GPU - OPTIMIZE 1
Input image shape: (968, 1428, 3)
Output image shape: (968, 1628, 3)
Total time: 78.079 seconds


#### 4.2.3. Đánh giá


Thời gian chạy khi thay đổi block size giữa các lần khác nhau chỉ có sự thay đổi ít. 
Đã có thử nghiệm với BLOCK_SIZE to hơn nhưng xảy ra lỗi khi thực hiện chạy chương trình.

Nguyên nhân: 
 - Có thể là do gpu đã xử lý gom thành các warp để chạy thread (1 thread chứa 32 warp)
 - Các GPU phần lớn hỗ trợ từ dưới 512 đến 1024 threads trong cùng 1 blocks, do đó nếu vượt số lượng threads tối đa sẽ lỗi.

Các thử nghiệm chi tiết hơn ở: https://colab.research.google.com/drive/1swEa2VcxwGrNvwCOXu3IzS_ixqWT1afK?usp=sharing

## 5. Nhìn lại quá trình làm đồ án

- Điểm tốt:
    - Nhóm học hỏi thêm được nhiều kiến thức về lập trình song song, numba, numpy và python
- Điểm chưa tốt: 
    - Trong quá trình làm việc nhóm nhóm còn có nhiều khuyết điểm dẫn đến làm chậm tiến độ làm việc
    - Nhóm dành thời gian quá nhiều vào phần so sánh output mà không tập trung vào tối ưu hóa song song
    - Bước phân tích cần làm tốt hơn
- Nếu có thời gian thêm:
    - Sửa lỗi làm sai lệch kết quả khi cài đặt SMEM cho hàm forward energy
    - Bổ sung báo cáo các bảng so sánh chi tiết hơn
    - Cài đặt song song cả hàm get_minimum_seam để giảm lượng dữ liệu phải transfer giữa host và device
    - Cải thiện thời gian chạy cho các phiên bản song song

## 6. Tài liệu tham khảo
- https://vnoi.info/wiki/cs/imageprocessing/Seam-Carving.md
- https://en.wikipedia.org/wiki/Seam_carving#cite_note-ACM1276390-1
- Forward energy: [Improved Seam Carving for Video Retargeting](https://faculty.idc.ac.il/arik/SCWeb/vidret/index.html)
- Song song
  - https://shwestrick.github.io/2020/07/29/seam-carve.html